In [8]:
import os
import httpx
from azure.identity import DefaultAzureCredential
from openai import AzureOpenAI
from PIL import Image
import json
from dotenv import load_dotenv

load_dotenv()

True

In [9]:
credential = DefaultAzureCredential()

In [10]:
client = AzureOpenAI(
    api_version="2024-02-01",
    azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT'),
    api_key=os.getenv('AZURE_OPENAI_API_KEY')
)

In [15]:
image_prompt = "chicago pd police car"

In [16]:
result = client.images.generate(
    model="dall-e-3", # the name of your DALL-E 3 deployment
    prompt=image_prompt,
    n=1
)

In [17]:
def count_files_in_directory(directory_path):
    try:
        # List only files, excluding directories
        file_count = sum(1 for entry in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, entry)))
        return file_count
    except FileNotFoundError:
        print("The directory does not exist.")
        return 0

In [18]:
fileNum = 0

# Set the directory for the stored image
image_dir = os.path.join(os.curdir, 'images')

# If the directory doesn't exist, create it
if not os.path.isdir(image_dir):
    os.mkdir(image_dir)

fileNum = count_files_in_directory(image_dir)

# Initialize the image path (note the filetype should be png)
image_path = os.path.join(image_dir, f'generated_image_{str(fileNum)}.png')

# Retrieve the generated image
json_response = json.loads(result.model_dump_json())
image_url = json_response["data"][0]["url"]  # extract image URL from response
generated_image = httpx.get(image_url).content  # download the image

with open(image_path, "wb") as image_file:
    image_file.write(generated_image)

# Display the image in the default image viewer
image = Image.open(image_path)
image.show()